In [4]:
import numpy as np
import pandas as pd

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

%matplotlib inline

# Load images and add labels

In [1]:
damsel_test_path = r'E:\processed_damsel_images\test\damsel_test.npy'
damsel_train_path = r'E:\processed_damsel_images\train\damsel_train.npy'

dragon_test_path = r'E:\processed_dragon_images\test\dragon_test.npy'
dragon_train_path = r'E:\processed_dragon_images\train\dragon_train.npy'

In [5]:
dragon_train_images = np.load(dragon_train_path)
dragon_test_path =  np.load(dragon_test_path)

MemoryError: Unable to allocate array with shape (1363476480,) and data type float64